# Hyperbolic Neural Net Messaround
Recently became fascinated with hyperbolic space and applications for NNs. Found the "hyperbolic learning library" (HypLL) and thought I'd give it a try

Started off with the the tutorial here https://hyperbolic-learning-library.readthedocs.io/en/latest/tutorials/cifar10_tutorial.html then tried some other experimentation

In [ ]:
from hypll.manifolds.poincare_ball import Curvature, PoincareBall

device = torch.device("cuda")

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)


batch_size = 4

trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2
)

testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2
)

classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

In [ ]:
from torch import nn

from hypll import nn as hnn


class Net(nn.Module):
    def __init__(self, manifold):
        super().__init__()
        self.conv1 = hnn.HConvolution2d(
            in_channels=3, out_channels=6, kernel_size=5, manifold=manifold
        )
        self.pool = hnn.HMaxPool2d(kernel_size=2, manifold=manifold, stride=2)
        self.conv2 = hnn.HConvolution2d(
            in_channels=6, out_channels=16, kernel_size=5, manifold=manifold
        )
        self.fc1 = hnn.HLinear(in_features=16 * 5 * 5, out_features=120, manifold=manifold)
        self.fc2 = hnn.HLinear(in_features=120, out_features=84, manifold=manifold)
        self.fc3 = hnn.HLinear(in_features=84, out_features=10, manifold=manifold)
        self.relu = hnn.HReLU(manifold=manifold)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.flatten(start_dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


criterion = nn.CrossEntropyLoss()

In [ ]:
from hypll.tensors import TangentTensor

def train(model, manifold, opt):
    for epoch in range(2):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)


            # move the inputs to the manifold
            tangents = TangentTensor(data=inputs, man_dim=1, manifold=manifold)
            manifold_inputs = manifold.expmap(tangents)

            # zero the parameter gradients
            opt.zero_grad()

            # forward + backward + optimize
            outputs = model(manifold_inputs)
            loss = criterion(outputs.tensor, labels)
            loss.backward()
            opt.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
                running_loss = 0.0

    print("Finished Training")

In [ ]:
# net.parameters() includes the learnable curvature "c" of the manifold.
from hypll.optim import RiemannianAdam

# Making the curvature a learnable parameter is usually suboptimal but can
# make training smoother.
manifold_poincare = PoincareBall(c=Curvature(requires_grad=True))
net_poincare = Net(manifold=manifold_poincare)
net_poincare.to(device)
optimizer_poincare = RiemannianAdam(net_poincare.parameters(), lr=0.001)

train(model=net_poincare, manifold=manifold_poincare, opt=optimizer_poincare)


This lib seems to allow swapping out the poincare manifold with euclidean manifold, does this act as a drop in replacement to convert to a "regular" euclidean net? Hopefully including the speedup..

In [ ]:
manifold_euclid = PoincareBall(c=Curvature(requires_grad=True))
net_euclid = Net(manifold=manifold_euclid)
net_euclid.to(device)
optimizer_poincare = RiemannianAdam(net_euclid.parameters(), lr=0.001)

train(model=net_euclid, manifold=manifold_euclid, opt=optimizer_poincare)

Seems to function as euclidean replacement, and is slightly faster than the hyperbolic version BUT nowhere near as fast as a traditional pytorch only euclidean version of the same model... I'm guessing because the hypll layers are written in pure pytorch, wheras pytorch native layers probably have some cuda kernels for speedup